In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from statsmodels.stats.multitest import multipletests
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import matplotlib
import pathlib
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from routines import save_figure, add_layout,\
    plot_unity, plot_regression, annotate_corr,\
    mhat, volcano,\
    get_sections, disjunction

# Load data

In [ ]:
path = 'path_to_data'
betas = pd.read_csv(f"{path}/betas.csv", index_col=0).T
pheno = pd.read_csv(f"{path}/pheno.csv", index_col=0)
pheno['Tissue'] = 'Blood WB'
df = pd.merge(pheno, betas, left_index=True, right_index=True)
manifest = pd.read_csv(f"data/850k.csv", index_col=0)
manifest['CHR'] = manifest['chr'].str[3::]

# Prepare data for Horvath's calculator

In [ ]:
pathlib.Path(f"{path}/calculator").mkdir(parents=True, exist_ok=True)
pheno.loc[:, ['Age', 'Female', 'Tissue']].to_csv(f"{path}/calculator/pheno.csv", na_rep="NA")
url_cpgs = "https://horvath.genetics.ucla.edu/html/dnamage/datMiniAnnotation.csv"
cpgs_calc = pd.read_csv(url_cpgs, index_col=0).index.values
cpgs_na = list(set(cpgs_calc) - set(betas.columns.values))
betas_calc = betas.loc[:, betas.columns.intersection(cpgs_calc)].copy()
betas_calc.loc[:, cpgs_na] = np.nan
betas_calc = betas_calc.astype('float32')
betas_calc = betas_calc.T
betas_calc.index.name = 'ProbeID'
betas_calc.to_csv(f"{path}/calculator/betas.csv", na_rep="NA")

# After that you need to upload these 2 files (pheno.csv and betas.csv) to Horvath's Online Calculator

# Load data from Horvath's calculator results

In [ ]:
calc = pd.read_csv(f"{path}/calculator/betas.output.csv", index_col=0)
feats = {
    'Hannum': 'Hannum',
    'Horvath': 'Horvath',
    'PhenoAge': 'PhenoAge',
    'GrimAge': 'GrimAge',
    'CD8T': 'CD8T',
    'CD4T': 'CD4T',
    'NK': 'NK',
    'Bcell': 'Bcell',
    'Mono': 'Mono',
    'Gran': 'Gran',
}
calc.rename(columns=feats, inplace=True)
samples = df.index.values
for feat_raw, feat_name in feats.items():
    df.loc[samples, feat_name] = calc.loc[samples, feat_name]

# Configuring experiments details:
## 1. Region-specific differences
## 2. Sex-Specific differences in Central
## 3. Sex-Specific differences in Yakutia

In [ ]:
experiments = {
    "region_specific": {
        "target": "Region",
        "path": "region_specific",
        "color": {
            "Central": "gold",
            "Yakutia": "lightslategray"
        },
        "color_line": "black",
        "filter": {
            "Central": (df["Region"] == "Central"),
            "Yakutia": (df["Region"] == "Yakutia")
        },
        "base_filter": (df["Region"] == "Central"),
        "base_part": "Central",
        "all_filter": (df["Region"].isin(["Central", "Yakutia"])),
    },
    "sex_specific_central": {
        "target": "Sex",
        "path": "sex_specific_central",
        "color": {
            "F": "hotpink",
            "M": "skyblue"
        },
        "color_line": "black",
        "filter": {
            "F": (df["Region"] == "Central") & (df["Sex"] == "F"),
            "M": (df["Region"] == "Central") & (df["Sex"] == "M")
        },
        "base_filter": (df["Region"] == "Central") & (df["Sex"] == "F"),
        "base_part": "F",
        "all_filter": (df["Region"] == "Central"),
    },
    "sex_specific_yakutia": {
        "target": "Sex",
        "path": "sex_specific_yakutia",
        "color": {
            "F": "firebrick",
            "M": "royalblue"
        },
        "color_line": 'black',
        "filter": {
            "F": (df["Region"] == "Yakutia") & (df["Sex"] == "F"),
            "M": (df["Region"] == "Yakutia") & (df["Sex"] == "M")
        },
        "base_filter": (df["Region"] == "Yakutia") & (df["Sex"] == "F"),
        "base_part": "F",
        "all_filter": (df["Region"] == "Yakutia"),
    },
}

# For each experiment we perform the following analysis and plots:

## 1. Plot samples histograms (distribution by age)

In [ ]:
hist_bins = np.linspace(5, 115, 23)
for experiment, exp_details in experiments.items():
    pathlib.Path(f"{path}/{exp_details['path']}/01_samples_hist").mkdir(parents=True, exist_ok=True)
    df_fig = df.loc[exp_details["all_filter"], ['Age', 'Sex', 'Region']].copy()
    df_fig.to_excel(f"{path}/{exp_details['path']}/01_samples_hist/fig.xlsx")
    dict_keys = {key: f"{key}: {df[exp_details['filter'][key]].shape[0]}" for key in exp_details['filter']}
    colors = {dict_keys[key]: val for key, val in exp_details['color'].items()}
    df_fig[exp_details['target']].replace(dict_keys, inplace=True)
    fig = plt.figure()
    sns.set_theme(style='whitegrid')
    hist = sns.histplot(
        data=df_fig,
        bins=hist_bins,
        edgecolor='k',
        linewidth=1,
        x="Age",
        hue=exp_details['target'],
        palette=colors
    )
    hist.set(xlim=(0, 120))
    plt.savefig(f"{path}/{exp_details['path']}/01_samples_hist/hist.png", bbox_inches='tight', dpi=400)
    plt.savefig(f"{path}/{exp_details['path']}/01_samples_hist/hist.pdf", bbox_inches='tight')
    plt.close(fig)

## 2. Cells distributions

In [ ]:
cells = ["CD8T", "CD4T", "NK", "Bcell", "Mono", "Gran"]
dist_num_bins = 15

for experiment, exp_details in experiments.items():
    pathlib.Path(f"{path}/{exp_details['path']}/02_cells").mkdir(parents=True, exist_ok=True)

    df_fig = df.loc[exp_details["all_filter"], cells + ["Sex", "Region", "Age"]]
    df_fig.to_excel(f"{path}/{exp_details['path']}/02_cells/fig.xlsx")

    df_stat = pd.DataFrame()
    for cell in cells:
        vals = {}
        for group in exp_details["filter"]:
            vals[group] = df.loc[exp_details["filter"][group], cell].values
            df_stat.at[cell, f"mean_{group}"] = np.mean(vals[group])
            df_stat.at[cell, f"median_{group}"] = np.median(vals[group])
            df_stat.at[cell, f"q75_{group}"], df_stat.at[cell, f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[cell, f"iqr_{group}"] = df_stat.at[cell, f"q75_{group}"] - df_stat.at[cell, f"q25_{group}"]
        _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
        df_stat.at[cell, "pval"] = pval

    _, df_stat["pval_fdr_bh"], _, _ = multipletests(df_stat["pval"], 0.05, method='fdr_bh')
    df_stat.to_excel(f"{path}/{exp_details['path']}/02_cells/stat.xlsx", index=True)

    for cell in cells:
        vals = {}
        for group in exp_details["filter"]:
            vals[group] = df.loc[exp_details["filter"][group], cell].values

        fig = go.Figure()
        for group_id, group in enumerate(exp_details["filter"]):
            if group_id == 0:
                pointpos = 1.5
            else:
                pointpos = -1.5
            fig.add_trace(
                go.Violin(
                    y=vals[group],
                    name=group,
                    box_visible=True,
                    meanline_visible=True,
                    showlegend=False,
                    line_color=exp_details["color_line"],
                    fillcolor=exp_details["color"][group],
                    marker=dict(color=exp_details["color"][group], line=dict(color=exp_details["color_line"],width=0.3), opacity=0.8),
                    points='all',
                    pointpos=pointpos,
                    bandwidth = np.ptp(vals[group]) / dist_num_bins,
                    opacity=0.8
                )
            )
        add_layout(fig, "", f"{cell}", f"p-value: {df_stat.at[cell, 'pval_fdr_bh']:0.2e}")
        fig.update_layout(title_xref='paper')
        fig.update_layout(legend_font_size=20)
        fig.update_xaxes(autorange=False, range=[-0.3, len(exp_details["filter"]) - 0.7])
        fig.update_layout(legend={'itemsizing': 'constant'})
        fig.update_layout(
            violingap=0.35,
            violingroupgap=0.35,
            width=800,
            height=600,
            margin=go.layout.Margin(
                l=120,
                r=50,
                b=70,
                t=50,
                pad=0,
            )
        )
        fig.update_layout(legend_y=1.01)
        save_figure(fig, f"{path}/{exp_details['path']}/02_cells/{cell}")

## 3. Epigenetic ages distributions

In [ ]:
ages = [
    "Hannum", "Horvath", "PhenoAge", "GrimAge",
    "PCHorvath1", "PCHorvath2", "PCHannum", "PCPhenoAge", "PCGrimAge",
]

dist_num_bins = 15

for experiment, exp_details in experiments.items():
    pathlib.Path(f"{path}/{exp_details['path']}/03_ages").mkdir(parents=True, exist_ok=True)

    df_fig = df.loc[exp_details["all_filter"], ["Age", "Sex", "Region"] + ages]
    df_fig_feats = ["Age"] + ages
    sns.set_theme(style="whitegrid", font_scale=1.8)
    pair_grid = sns.PairGrid(
        data=df_fig,
        vars=df_fig_feats,
        hue=exp_details["target"],
        hue_order=list(exp_details["color"].keys()),
        palette=exp_details["color"]
    )
    pair_grid.map_lower(plot_unity)
    pair_grid.map_lower(sns.scatterplot, s=35, alpha=0.75, linewidth=0)
    pair_grid.map_diag(sns.histplot, bins=np.linspace(5, 115, 23))
    pair_grid.map_lower(
        plot_regression,
        base_indexes=df.index[exp_details["base_filter"]],
        base_color=exp_details["color"][exp_details["base_part"]],
        bkg_color=exp_details["color_line"]
    )
    pair_grid.map_upper(
        annotate_corr,
        base_indexes=df.index[exp_details["base_filter"]],
        colors=list(exp_details["color"].values()),
        bkg_color=exp_details["color_line"]
    )
    for x_axis_id in range(pair_grid.axes.shape[0]):
        for y_axis_id in range(pair_grid.axes.shape[1]):
            pair_grid.axes[x_axis_id, y_axis_id].spines[['right', 'top']].set_visible(True)
            if x_axis_id != y_axis_id:
                pass
            if x_axis_id <= y_axis_id:
                pair_grid.axes[x_axis_id, y_axis_id].grid(False)

    plt.savefig(f"{path}/{exp_details['path']}/03_ages/scatter_mtx.png", bbox_inches='tight', dpi=200)
    plt.savefig(f"{path}/{exp_details['path']}/03_ages/scatter_mtx.pdf", bbox_inches='tight')
    plt.clf()

    df_stat = pd.DataFrame(index=[f"{x}Acc" for x in ages], columns=["pval", "pval_fdr_bh"])
    for age in ages:
        formula = f"{age} ~ Age"
        model = smf.ols(formula=formula, data=df.loc[exp_details["base_filter"], :]).fit()
        df[f"{age}_linear_pred"] = model.predict(df)
        df[f"{age}Acc"] = df[age] - df[f"{age}_linear_pred"]

        vals = {}
        for group in exp_details["filter"]:
            vals[group] = df.loc[exp_details["filter"][group], f"{age}Acc"].values
            df_stat.at[f"{age}Acc", f"mean_{group}"] = np.mean(vals[group])
            df_stat.at[f"{age}Acc", f"median_{group}"] = np.median(vals[group])
            df_stat.at[f"{age}Acc", f"q75_{group}"], df_stat.at[f"{age}Acc", f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
            df_stat.at[f"{age}Acc", f"iqr_{group}"] = df_stat.at[f"{age}Acc", f"q75_{group}"] - df_stat.at[f"{age}Acc", f"q25_{group}"]

        _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
        df_stat.at[f"{age}Acc", "pval"] = pval

    _, df_stat["pval_fdr_bh"], _, _ = multipletests(df_stat["pval"], 0.05, method='fdr_bh')
    df_stat.to_excel(f"{path}/{exp_details['path']}/03_ages/stat.xlsx", index=True)

    df_fig = df.loc[exp_details["all_filter"], ["Sex", "Region", "Age"] + ages + [f"{x}Acc" for x in ages]]
    df_fig.to_excel(f"{path}/{exp_details['path']}/03_ages/fig.xlsx")

    for age in ages:
        vals = {}
        for group in exp_details["filter"]:
            vals[group] = df.loc[exp_details["filter"][group], f"{age}Acc"].values

        fig = go.Figure()
        for group_id, group in enumerate(exp_details["filter"]):
            if group_id == 0:
                pointpos = 1.5
            else:
                pointpos = -1.5

            fig.add_trace(
                go.Violin(
                    y=vals[group],
                    name=group,
                    box_visible=True,
                    meanline_visible=True,
                    showlegend=False,
                    line_color=exp_details["color_line"],
                    fillcolor=exp_details["color"][group],
                    marker=dict(color=exp_details["color"][group], line=dict(color=exp_details["color_line"],width=0.3), opacity=0.8),
                    points='all',
                    pointpos=pointpos,
                    bandwidth=np.ptp(vals[group]) / dist_num_bins,
                    opacity=0.8,
                )
            )
        add_layout(fig, "", f"{age}Acc", f"p-value: {df_stat.at[f'{age}Acc', 'pval_fdr_bh']:0.2e}")
        fig.update_layout(title_xref='paper')
        fig.update_layout(legend_font_size=20)
        fig.update_xaxes(autorange=False, range=[-0.3, len(exp_details["filter"]) - 0.7])
        fig.update_layout(legend= {'itemsizing': 'constant'})
        fig.update_layout(
            violingap=0.35,
            violingroupgap=0.35,
            width=500,
            height=600,
            margin=go.layout.Margin(
                l=100,
                r=50,
                b=50,
                t=50,
                pad=0,
            )
        )
        fig.update_layout(legend_y=1.01)
        save_figure(fig, f"{path}/{exp_details['path']}/03_ages/violin_{age}Acc")

        min_val = df[["Age", age]].min().min()
        max_val = df[["Age", age]].max().max()
        shift_val = max_val - min_val
        min_val -= 0.05 * shift_val
        max_val += 0.05 * shift_val

        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=[min_val, max_val],
                y=[min_val, max_val],
                showlegend=False,
                name="",
                mode="lines",
                marker_color="black",
                marker=dict(
                    size=8,
                    opacity=0.75,
                    line=dict(
                        color="black",
                        width=0.5
                    )
                )
            )
        )
        fig.add_trace(
            go.Scatter(
                x=df.loc[exp_details["base_filter"], f"Age"].values,
                y=df.loc[exp_details["base_filter"], f"{age}_linear_pred"].values,
                showlegend=False,
                name="",
                mode="lines",
                line=dict(width=5),
                marker_color=exp_details["color"][exp_details["base_part"]],
                marker=dict(
                    size=8,
                    opacity=0.75,
                    line=dict(
                        color="black",
                        width=0.5
                    )
                )
            )
        )
        for group in exp_details["filter"]:
            fig.add_trace(
                go.Scatter(
                    x=df.loc[exp_details["filter"][group], f"Age"].values,
                    y=df.loc[exp_details["filter"][group], f"{age}"].values,
                    showlegend=True,
                    name=group,
                    mode="markers",
                    line_color=exp_details["color"][group],
                    marker=dict(
                        size=8,
                        opacity=0.75,
                        line=dict(
                            color=exp_details["color_line"],
                            width=0.5
                        )
                    )
                )
            )
        add_layout(fig, f"Age", f"{age}", f"")
        fig.update_layout(legend_font_size=20)
        fig.update_layout(legend= {'itemsizing': 'constant'})
        fig.update_xaxes(autorange=False)
        fig.update_yaxes(autorange=False)
        fig.update_layout(title_xref='paper')
        fig.update_layout(xaxis_range=[min_val, max_val])
        fig.update_layout(yaxis_range=[min_val, max_val])
        fig.update_layout(
            width=850,
            height=800,
            margin=go.layout.Margin(
                l=100,
                r=50,
                b=100,
                t=50,
                pad=0,
            )
        )
        save_figure(fig, f"{path}/{exp_details['path']}/03_ages/scatter_{age}")

## 4. DunedinPACE distribution

In [ ]:
dist_num_bins = 15

for experiment, exp_details in experiments.items():
    pathlib.Path(f"{path}/{exp_details['path']}/04_dunedin_pace").mkdir(parents=True, exist_ok=True)

    df_fig = df.loc[exp_details["all_filter"], ["Sex", "Region", "Age", "DunedinPACE"]]
    df_fig.to_excel(f"{path}/{exp_details['path']}/04_dunedin_pace/fig.xlsx")

    df_stat = pd.DataFrame()
    vals = {}
    for group in exp_details["filter"]:
        vals[group] = df.loc[exp_details["filter"][group], "DunedinPACE"].values
        df_stat.at["DunedinPACE", f"mean_{group}"] = np.mean(vals[group])
        df_stat.at["DunedinPACE", f"median_{group}"] = np.median(vals[group])
        df_stat.at["DunedinPACE", f"q75_{group}"], df_stat.at["DunedinPACE", f"q25_{group}"] = np.percentile(vals[group], [75 , 25])
        df_stat.at["DunedinPACE", f"iqr_{group}"] = df_stat.at["DunedinPACE", f"q75_{group}"] - df_stat.at["DunedinPACE", f"q25_{group}"]
    _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
    df_stat.at["DunedinPACE", "pval"] = pval
    df_stat.to_excel(f"{path}/{exp_details['path']}/04_dunedin_pace/stat.xlsx", index=True)

    fig = go.Figure()
    for group_id, group in enumerate(exp_details["filter"]):
        if group_id == 0:
            pointpos = 1.5
        else:
            pointpos = -1.5
        fig.add_trace(
            go.Violin(
                y=vals[group],
                name=group,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color=exp_details["color_line"],
                fillcolor=exp_details["color"][group],
                marker=dict(color=exp_details["color"][group], line=dict(color=exp_details["color_line"],width=0.3), opacity=0.8),
                points='all',
                pointpos=pointpos,
                bandwidth = np.ptp(vals[group]) / dist_num_bins,
                opacity=0.8
            )
        )
    add_layout(fig, "", f"DunedinPACE", f"p-value: {df_stat.at['DunedinPACE', 'pval']:0.2e}")
    fig.update_layout(title_xref='paper')
    fig.update_layout(legend_font_size=20)
    fig.update_xaxes(autorange=False, range=[-0.3, len(exp_details["filter"]) - 0.7])
    fig.update_layout(legend={'itemsizing': 'constant'})
    fig.update_layout(
        violingap=0.35,
        violingroupgap=0.35,
        width=500,
        height=600,
        margin=go.layout.Margin(
            l=100,
            r=50,
            b=50,
            t=50,
            pad=0,
        )
    )
    fig.update_layout(legend_y=1.01)
    save_figure(fig, f"{path}/{exp_details['path']}/04_dunedin_pace/violin")

## 5. DMP analysis

In [ ]:
n_highlights = 2
n_examples = 10
dist_num_bins = 15

reg_enr_orders = {
    'CHR': [str(x) for x in range(1, 24)],
    'RELATION_TO_UCSC_CPG_ISLAND': ['S_Shelf', 'S_Shore', 'Island', 'N_Shore', 'N_Shelf', 'OpenSea'],
    'UCSC_REFGENE_GROUP': ['TSS1500', 'TSS200', '5\'UTR', '1stExon', 'Body', '3\'UTR']
}
reg_enr_col_names = {
    'CHR': "CHR",
    'RELATION_TO_UCSC_CPG_ISLAND': "Relation_to_Island",
    'UCSC_REFGENE_GROUP': "UCSC_RefGene_Group"
}
reg_enr_fig_sizes = {
    'CHR': (17, 10),
    'RELATION_TO_UCSC_CPG_ISLAND': (5, 10),
    'UCSC_REFGENE_GROUP': (5, 10)
}
reg_enr_colors = {
    'CHR': px.colors.qualitative.Dark24,
    'RELATION_TO_UCSC_CPG_ISLAND': px.colors.qualitative.Light24[17:23],
    'UCSC_REFGENE_GROUP': px.colors.qualitative.Light24[11:17]
}

for experiment, exp_details in experiments.items():
    path_dmp = f"{path}/{exp_details['path']}/05_DMP"
    pathlib.Path(f"{path_dmp}/examples").mkdir(parents=True, exist_ok=True)
    pathlib.Path(f"{path_dmp}/reg_enr").mkdir(parents=True, exist_ok=True)

    df_dmps = pd.read_csv(f"{path}/{exp_details['path']}/DMP.csv", index_col=0)
    df_dmps["CpG"] = df_dmps.index.values
    df_dmps.sort_values(["adj.P.Val"], ascending=[True], inplace=True)
    df_dmps['print'] = df_dmps.apply(lambda row: f"{row['CpG']} ({row['gene']})", axis=1)
    top_to_hightlight = df_dmps["print"].values[0:n_highlights]
    df_dmps['log_pval'] = -np.log10(df_dmps["adj.P.Val"])
    sns.set_theme(style='whitegrid')
    df_dmps.sort_values(["MAPINFO"], ascending=[True], inplace=True)
    mhat(
        df=df_dmps,
        chr='CHR',
        pv='adj.P.Val',
        path=f"{path_dmp}",
        valpha=1,
        markernames=tuple(top_to_hightlight),
        markeridcol='print',
        gstyle=2,
        dim=(8, 4),
        axtickfontsize=8
    )
    sns.set_theme(style='whitegrid')
    volcano(
        df=df_dmps,
        lfc='logFC',
        pv='adj.P.Val',
        pv_thr=(1, 1),
        lfc_thr=(0.0, 0.0),
        path=f"{path_dmp}",
        genenames=tuple(top_to_hightlight),
        geneid='print',
        gstyle=2,
        sign_line=False,
        color=(list(exp_details["color"].values())[1], "lavender", list(exp_details["color"].values())[0]),
        dim=(4, 4)
    )

    df_dmps.sort_values(["P.Value"], ascending=[True], inplace=True)
    df_dmps_selected = df_dmps.head(1000)
    df_dmps_selected.to_excel(f"{path_dmp}/cpgs.xlsx")
    print(f"Number of CpGs: {df_dmps_selected.shape[0]}")

    dmps_genes = set()
    for cpg in df_dmps_selected.index.values:
        genes_raw = manifest.at[cpg, 'Gene']
        if isinstance(genes_raw, str):
            genes = genes_raw.split(';')
            dmps_genes.update(set(genes))
    if 'non-genic' in dmps_genes:
        dmps_genes.remove('non-genic')
    if ' ' in dmps_genes:
        dmps_genes.remove(' ')
    dmps_genes = list(dmps_genes)
    df_dmps_genes = pd.DataFrame({'gene': dmps_genes})
    df_dmps_genes.to_excel(f"{path_dmp}/genes.xlsx", index=False)
    print(f"Number of genes: {df_dmps_genes.shape[0]}")

    df_dmps_examples = df_dmps_selected.sort_values(['adj.P.Val'], ascending=[True]).head(n_examples)
    for cpg_id, (cpg, row) in enumerate(df_dmps_examples.iterrows()):
        pval = row['adj.P.Val']
        log_fc = row['logFC']
        gene = manifest.at[cpg, 'Gene']

        fig = go.Figure()
        for group_id, group in enumerate(exp_details["filter"]):
            if group_id == 0:
                pointpos = 1.5
            else:
                pointpos = -1.5
            vals = df.loc[exp_details["filter"][group], cpg].values
            fig.add_trace(
                go.Violin(
                    y=vals,
                    name=group,
                    box_visible=True,
                    meanline_visible=True,
                    showlegend=False,
                    line_color='black',
                    fillcolor=exp_details["color"][group],
                    marker = dict(color=exp_details["color"][group], line=dict(color='black',width=0.3), opacity=0.8),
                    points='all',
                    pointpos=pointpos,
                    bandwidth = np.ptp(vals) / dist_num_bins,
                    opacity=0.8
                )
            )
        add_layout(fig, "", "Methylation", f"{cpg} ({gene})<br>p-value: {pval:0.2e}<br>log(Fold Change): {log_fc:0.2e}")
        fig.update_layout(title_xref='paper', title={'y': 0.95})
        fig.update_layout(legend_font_size=20)
        fig.update_xaxes(autorange=False, range=[-0.3, len(exp_details["filter"]) - 0.7])
        fig.update_layout(legend={'itemsizing': 'constant'})
        fig.update_layout(
            violingap=0.35,
            violingroupgap=0.35,
            width=850,
            height=615,
            margin=go.layout.Margin(
                l=120,
                r=50,
                b=90,
                t=120,
                pad=0,
            )
        )
        save_figure(fig, f"{path_dmp}/examples/{cpg_id}_{cpg}")
    df_fig = df.loc[:, ["Age", "Sex", "Region"] + list(df_dmps_examples.index.values)]
    df_fig.to_excel(f"{path_dmp}/examples/fig.xlsx")

    df_dmps_fisher_target = manifest.loc[df_dmps_selected.index.values, :]
    df_dmps_fisher_global = manifest.loc[df_dmps.index.values, :]
    df_dmps_fisher_padding = df_dmps_fisher_global.loc[~df_dmps_fisher_global.index.isin(df_dmps_selected.index.values), :]
    for var in reg_enr_orders:
        columns=["11", "12", "21", "22", "sum", "pval", "odds_ratio"]
        df_var = pd.DataFrame(index=reg_enr_orders[var], columns=columns, data=np.zeros((len(reg_enr_orders[var]), len(columns))))
        df_var.index.name = reg_enr_col_names[var].replace("_", " ")
        for var_val in reg_enr_orders[var]:
            contingency_table = pd.DataFrame(index=["specific", "non-specific"], columns=["in_val", "not_in_val"])
            contingency_table.at["specific", "in_val"] = df_dmps_fisher_target.loc[df_dmps_fisher_target[reg_enr_col_names[var]] == var_val, :].shape[0]
            contingency_table.at["specific", "not_in_val"] = df_dmps_fisher_target.loc[df_dmps_fisher_target[reg_enr_col_names[var]] != var_val, :].shape[0]
            contingency_table.at["non-specific", "in_val"] = df_dmps_fisher_padding.loc[df_dmps_fisher_padding[reg_enr_col_names[var]] == var_val, :].shape[0]
            contingency_table.at["non-specific", "not_in_val"] = df_dmps_fisher_padding.loc[df_dmps_fisher_padding[reg_enr_col_names[var]] != var_val, :].shape[0]
            df_var.at[var_val, "11"] = contingency_table.at["specific", "in_val"]
            df_var.at[var_val, "12"] = contingency_table.at["specific", "not_in_val"]
            df_var.at[var_val, "21"] = contingency_table.at["non-specific", "in_val"]
            df_var.at[var_val, "22"] = contingency_table.at["non-specific", "not_in_val"]
            df_var.at[var_val, "sum"] = contingency_table.values.sum()
            odds_ratio, pval = stats.fisher_exact(contingency_table.to_numpy(), alternative='two-sided')
            if np.isnan(odds_ratio):
                odds_ratio = 1.0
            df_var.at[var_val, "odds_ratio"], df_var.at[var_val, "pval"] = odds_ratio, pval
        _, df_var['pval_fdr_bh'], _, _ = multipletests(df_var['pval'].values, 0.05, method='fdr_bh')
        df_var[r'$ \log_{10}(\mathrm{Odds\ ratio})$'] = np.log10(df_var.loc[:, 'odds_ratio'].values)
        df_var[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(df_var.loc[:, 'pval_fdr_bh'].values)

        plt.figure(figsize=reg_enr_fig_sizes[var])
        plt.xticks(rotation=90)
        sns.set_theme(style='whitegrid', font_scale=2)
        cmap = plt.get_cmap("viridis").copy()
        cmap.set_under('black')

        plot = plt.scatter(
            df_var.index,
            df_var.loc[:, r'$ -\log_{10}(\mathrm{p-value})$'].values,
            c=df_var.loc[:, r'$ -\log_{10}(\mathrm{p-value})$'].values,
            cmap=cmap,
            vmin=-np.log10(0.05)
        )
        plt.clf()
        cbar = plt.colorbar(plot, extend='min')

        df_var['bar_color'] = 'black'
        for df_var_index in df_var.index.values:
            if df_var.at[df_var_index, "pval_fdr_bh"] < 0.05:
                value_tmp = df_var.at[df_var_index, r'$ -\log_{10}(\mathrm{p-value})$']
                value_color = (value_tmp-cbar.vmin)/(cbar.vmax-cbar.vmin)
                df_var.at[df_var_index, 'bar_color'] = matplotlib.colors.rgb2hex(cbar.cmap(value_color))
        df_var.to_excel(f"{path_dmp}/reg_enr/fisher_{var}.xlsx")

        plt.xticks(rotation=90)
        cbar.set_label(r"$-\log_{10}(\mathrm{p-value})$", horizontalalignment='center')
        ax = sns.barplot(
            data=df_var,
            x=df_var.index,
            y=r'$ \log_{10}(\mathrm{Odds\ ratio})$',
            palette=df_var.loc[:, 'bar_color'],
            dodge=False,
            edgecolor='black',
        )
        plt.savefig(f"{path_dmp}/reg_enr/fisher_{var}.png", bbox_inches='tight')
        plt.savefig(f"{path_dmp}/reg_enr/fisher_{var}.pdf", bbox_inches='tight')
        plt.close()

# Region-specific summary

In [ ]:
path_local = f"{path}/{experiments['region_specific']['path']}/05_DMP"
genes_our = pd.read_excel(f"{path_local}/genes.xlsx", index_col='gene').index.values
genes_ref = pd.read_excel(f"data/region_specific_lists/Cardona2014.xlsx", index_col='gene').index.values

pathlib.Path(f"{path_local}/genes_intersection").mkdir(parents=True, exist_ok=True)

fig, ax = plt.subplots()
venn = venn2(
    subsets=(set(genes_our), set(genes_ref)),
    set_labels = ('DMPs', 'Cardona2014'),
    set_colors=('r', 'g'),
    alpha = 0.5
)
venn2_circles(subsets=(set(genes_our), set(genes_ref)))
for text in venn.set_labels:
    text.set_fontsize(16)
for text in venn.subset_labels:
    text.set_fontsize(25)
plt.savefig(f"{path_local}/genes_intersection/venn.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path_local}/genes_intersection/venn.pdf", bbox_inches='tight')
plt.clf()

sections = get_sections([set(genes_our), set(genes_ref)])
for sec in sections:
    df_sec = pd.DataFrame(index=list(sections[sec]))
    df_sec.to_excel(f"{path_local}/genes_intersection/{sec}.xlsx", index_label='gene')

df_genes = pd.DataFrame(index=genes_our)
df_genes["Cardona et. al. 2014"] = "No"
df_genes.loc[set(genes_our).intersection(set(genes_ref)), "Cardona et. al. 2014"] = "Yes"
df_genes.to_excel(f"{path_local}/genes_intersection/genes.xlsx", index_label='gene')

df_cpgs = pd.read_excel(f"{path_local}/cpgs.xlsx", index_col=0)
dict_col = {
    "logFC": "logFC",
    "adj.P.Val": "Adj. p-value",
    "Central_AVG": "Central avg",
    "Yakutia_AVG": "Yakutia avg",
    "deltaBeta": "Delta"
}
df_cpgs.rename(columns=dict_col, inplace=True)
df_cpgs.loc[:, "Gene"] = manifest.loc[df_cpgs.index.values, 'Gene']
df_cpgs.loc[:, "Relation to Island"] = manifest.loc[df_cpgs.index.values, 'Relation_to_Island']
df_cpgs.loc[:, "UCSC RefGene Group"] = manifest.loc[df_cpgs.index.values, 'UCSC_RefGene_Group']

df_cpgs = df_cpgs.loc[:, ["CHR", "MAPINFO", "Gene", "Relation to Island", "UCSC RefGene Group", "logFC", "Central avg", "Yakutia avg", "Delta"]]
df_cpgs.to_excel(f"{path_local}/cpgs_processed.xlsx", index_label='CpG')

## Sex-specific summary

In [ ]:
path_local = f"{path}/sex_specificity_in_regions"
df_cpgs_ctl = pd.read_csv(f"{path}/{experiments['sex_specific_central']['path']}/DMP.csv", index_col=0)
cpgs_ctl = pd.read_excel(f"{path}/{experiments['sex_specific_central']['path']}/05_DMP/cpgs.xlsx", index_col=0).index.values
df_cpgs_ctl["Significant in Central"] = "No"
df_cpgs_ctl.loc[cpgs_ctl, "Significant in Central"] = "Yes"
dict_col_ctl = {
    "logFC": "logFC in Central",
    "adj.P.Val": "Adj. p-value in Central",
    "F_AVG": "F avg in Central",
    "M_AVG": "M avg in Central",
    "deltaBeta": "Delta in Central"
}
df_cpgs_ctl.rename(columns=dict_col_ctl, inplace=True)

df_cpgs_ykt = pd.read_csv(f"{path}/{experiments['sex_specific_yakutia']['path']}/DMP.csv", index_col=0)
cpgs_ykt = pd.read_excel(f"{path}/{experiments['sex_specific_yakutia']['path']}/05_DMP/cpgs.xlsx", index_col=0).index.values
df_cpgs_ykt["Significant in Yakutia"] = "No"
df_cpgs_ykt.loc[cpgs_ykt, "Significant in Yakutia"] = "Yes"
dict_col_ykt = {
    "logFC": "logFC in Yakutia",
    "adj.P.Val": "Adj. p-value in Yakutia",
    "F_AVG": "F avg in Yakutia",
    "M_AVG": "M avg in Yakutia",
    "deltaBeta": "Delta in Yakutia"
}
df_cpgs_ykt.rename(columns=dict_col_ykt, inplace=True)

cpgs_grant2022 = pd.read_excel(f"data/sex_specific_lists/Grant2022.xlsx", index_col='CpG').index.values
cpgs_inoshita2015 = pd.read_excel(f"data/sex_specific_lists/Inoshita2015.xlsx", index_col='CpG').index.values
cpgs_mccarthy2014 = pd.read_excel(f"data/sex_specific_lists/McCarthy2014.xlsx", index_col='CpG').index.values

df_cpgs_cmn = df_cpgs_ctl.loc[:, ["CHR", "MAPINFO"]]
cpgs_cmn = df_cpgs_cmn.index.values
df_cpgs_cmn.loc[cpgs_cmn, "Gene"] = manifest.loc[cpgs_cmn, 'Gene']
df_cpgs_cmn.loc[cpgs_cmn, "Relation to Island"] = manifest.loc[cpgs_cmn, 'Relation_to_Island']
df_cpgs_cmn.loc[cpgs_cmn, "UCSC RefGene Group"] = manifest.loc[cpgs_cmn, 'UCSC_RefGene_Group']

df_cpgs_cmn.loc[cpgs_cmn, "logFC in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "logFC in Central"]
df_cpgs_cmn.loc[cpgs_cmn, "Adj. p-value in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "Adj. p-value in Central"]
df_cpgs_cmn.loc[cpgs_cmn, "F avg in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "F avg in Central"]
df_cpgs_cmn.loc[cpgs_cmn, "M avg in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "M avg in Central"]
df_cpgs_cmn.loc[cpgs_cmn, "Delta in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "Delta in Central"]
df_cpgs_cmn.loc[cpgs_cmn, "Significant in Central"] = df_cpgs_ctl.loc[cpgs_cmn, "Significant in Central"]

df_cpgs_cmn.loc[cpgs_cmn, "logFC in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "logFC in Yakutia"]
df_cpgs_cmn.loc[cpgs_cmn, "Adj. p-value in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "Adj. p-value in Yakutia"]
df_cpgs_cmn.loc[cpgs_cmn, "F avg in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "F avg in Yakutia"]
df_cpgs_cmn.loc[cpgs_cmn, "M avg in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "M avg in Yakutia"]
df_cpgs_cmn.loc[cpgs_cmn, "Delta in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "Delta in Yakutia"]
df_cpgs_cmn.loc[cpgs_cmn, "Significant in Yakutia"] = df_cpgs_ykt.loc[cpgs_cmn, "Significant in Yakutia"]

df_cpgs_cmn["Grant et. al. 2022"] = "No"
df_cpgs_cmn.loc[set(cpgs_cmn).intersection(set(cpgs_grant2022)), "Grant et. al. 2022"] = "Yes"

df_cpgs_cmn["Inoshita et. al. 2015"] = "No"
df_cpgs_cmn.loc[set(cpgs_cmn).intersection(set(cpgs_inoshita2015)), "Inoshita et. al. 2015"] = "Yes"

df_cpgs_cmn["McCarthy et. al. 2014"] = "No"
df_cpgs_cmn.loc[set(cpgs_cmn).intersection(set(cpgs_mccarthy2014)), "McCarthy et. al. 2014"] = "Yes"

pathlib.Path(f"{path_local}/cpgs").mkdir(parents=True, exist_ok=True)
conds_cols = [
    "Significant in Central",
    "Significant in Yakutia",
    "Grant et. al. 2022",
    "Inoshita et. al. 2015",
    "McCarthy et. al. 2014"
]
df_intxn_order = pd.DataFrame(index=conds_cols)
df_intxn_order.to_excel(f"{path_local}/cpgs/conds_cols.xlsx", index_label='Set')
conditions = [df_cpgs_cmn[metric] == "Yes"  for metric in ["Significant in Central", "Significant in Yakutia"]]
df_cpgs_cmn = df_cpgs_cmn[disjunction(conditions)]
df_cpgs_cmn.to_excel(f"{path_local}/cpgs/table.xlsx", index_label='CpG')

sections = get_sections([set(cpgs_ctl), set(cpgs_ykt), set(cpgs_grant2022), set(cpgs_inoshita2015), set(cpgs_mccarthy2014)])
for sec in sections:
    df_sec = pd.DataFrame(index=list(sections[sec]))
    df_sec.to_excel(f"{path_local}/cpgs/{sec}.xlsx", index_label='gene')

dict_upset_lists = {
    "McCarthy et. al. 2014": cpgs_mccarthy2014,
    "Inoshita et. al. 2015": cpgs_inoshita2015,
    "Grant et. al. 2022": cpgs_grant2022,
    'Sex-specific in Yakutia': cpgs_ykt,
    'Sex-specific in Central': cpgs_ctl,
}
upset_all = list(set().union(*list(dict_upset_lists.values())))
df_upset = pd.DataFrame(index=upset_all)
for k, v in dict_upset_lists.items():
    df_upset[k] = df_upset.index.isin(v)
df_upset = df_upset.set_index(list(dict_upset_lists.keys()))
tmp = plt.figure(figsize=(32, 8))
upset_fig = upsetplot.UpSet(
    df_upset,
    sort_categories_by='input',
    subset_size='count',
    show_counts=True,
    min_degree=0,
    element_size=None,
    totals_plot_elements=3,
    include_empty_subsets=False
)
upset_fig.style_subsets(present=["Sex-specific in Central", "Sex-specific in Yakutia"], edgecolor="red", linewidth=2)
upset_fig.style_subsets(present=["Sex-specific in Central", "Grant et. al. 2022"], absent=["Sex-specific in Yakutia"], facecolor="blue")
upset_fig.style_subsets(present=["Sex-specific in Yakutia", "Grant et. al. 2022"], absent=["Sex-specific in Central"], facecolor="green")
upset_fig.style_subsets(present=["Sex-specific in Yakutia", "Sex-specific in Central", "Grant et. al. 2022"], facecolor="yellow")
upset_fig.plot(tmp)
plt.savefig(f"{path_local}/cpgs/upset.png", bbox_inches='tight')
plt.savefig(f"{path_local}/cpgs/upset.pdf", bbox_inches='tight')
plt.close()